Certainty factors are numerical values in the range $[-1, 1]$ that Emycin uses to represent boolean values with associated confidence. Negative certainty factors represent False values, with increasing confidence as the number approaches -1.0. Similarly, positive CFs represent True, with increasing confidence approaching 1.0. A CF of 0.0 represents Unknown.

We can combine certainty factors in a manner similar to boolean logic using $AND$ and $OR$ operations; however, since CFs are real numbers, we can't use the standard truth tables of boolean logic. These definitions come from Peter Norvig's PAIP.

In [15]:
data CF = CF {true :: Float, false :: Float, unknown :: Float, cutoff :: Float} deriving (Show)

In [16]:
currCf = CF {true = 1.0, false = -1.0, unknown = 0.0, cutoff = 0.2}

In [3]:
cfOr a b 
        | a > 0 && b > 0 =  a + b - a * b
        | a < 0 && b < 0 =  a + b + a * b
        | otherwise      =  quot (a + b) (1 - minimum [abs a, abs b] )

In [4]:
cfAnd a b = minimum [a, b]

In [19]:
isCf x = false currCf <= x && x <= true currCf

In [20]:
cfTrue x = isCf x && x > cutoff currCf

In [22]:
cfFalse x = isCf x && x < (cutoff currCf - 1)

Since Emycin aims to provide a flexible framework adaptable to varied problem domains, its representation of "types" needs to be extensible. An expert should be able to define the types of things about which the system reasons. Emycin calls these types contexts, and specific things in the system are represented with instances of contexts.

In [27]:
data Context = Context { count :: Int, -- # track Instances with numerical IDs
        name :: String,
        initialData :: [String], -- params to find out before reasoning
        goals :: [String]} deriving (Show) -- params to find out during reasoning}
        

In [28]:
increaseCount :: Context -> Context
increaseCount Context {count = c, name = n, initialData = i, goals = g} = Context {count = c + 1, name = n, initialData = i, goals = g}

In [29]:
instantiate :: Context -> (String, Int)
instantiate c = (name c, count c)

Contexts need attributes so that individual instances can be differentiated and tested by the reasoner. Emycin represents attributes of contexts Parameters, and instances have a value for each of the parameters of its context. These are defined by the expert for each context in the problem domain.

- ctx: The Context to which this Parameter is associated.
- enum: If specified, indicates that values of this parameter must be members of the given list of values.
- cls: If specified, indicates that values of this parameter must be instances of the given type.
- ask_first: If True, to determine a value of this parameter, first ask the user before reasoning.

In [ ]:
-- data Parameter = Parameter {name = name, ctx = ctx, enum = enum, ask_first = ask_first, cls = cls}

In [8]:

data Fact = Fact { name :: String, value :: String } deriving (Show, Eq)
data Rule = Rule { number :: Int, conditions :: [Fact], conclusion :: String } deriving (Show)

-- data Fact = Fact { name :: String, value :: String } deriving (Show, Eq)

type WorkingMemory = [Fact]

buildWorkingMemory :: IO WorkingMemory
buildWorkingMemory = do
  putStrLn "Enter the number of facts to add to working memory: "
  numFacts <- readLn :: IO Int
  if numFacts == 0
    then return []
    else do
      putStrLn "Enter facts in the format 'name:value', one per line: "
      facts <- sequence $ replicate numFacts (readFact)
      return facts

readFact :: IO Fact
readFact = do
  input <- getLine
  let (name, value) = break (== ':') input
  return (Fact { name = name, value = tail value })


Line 15: Redundant return
Found:
do putStrLn
     "Enter facts in the format 'name:value', one per line: "
   facts <- sequence $ replicate numFacts (readFact)
   return facts
Why not:
do putStrLn
     "Enter facts in the format 'name:value', one per line: "
   sequence $ replicate numFacts (readFact)Line 17: Use replicateM
Found:
sequence $ replicate numFacts (readFact)
Why not:
Control.Monad.replicateM numFacts readFactLine 17: Redundant bracket
Found:
(readFact)
Why not:
readFact

In [13]:
buildWorkingMemory

: 

In [9]:
-- module Mycin where
-- type WorkingMemory = [Fact]

checkCondition :: WorkingMemory -> Fact -> Bool
checkCondition wm (Fact name value) = any (\(Fact n v) -> n == name && v == value) wm

checkConditions :: WorkingMemory -> [Fact] -> Bool
checkConditions wm = all (checkCondition wm)

applyRule :: Rule -> WorkingMemory -> Maybe Fact
applyRule rule wm =
  if checkConditions wm (conditions rule)
    then Just $ Fact "diagnosis" (conclusion rule)
    else Nothing

findApplicableRules :: [Rule] -> WorkingMemory -> [Rule]
findApplicableRules rules wm = filter (\r -> applyRule r wm /= Nothing) rules

inferenceEngine :: [Rule] -> WorkingMemory -> Maybe Fact
inferenceEngine rules wm = case findApplicableRules rules wm of
  [] -> Nothing
  applicableRules -> applyRule (last applicableRules) wm

diagnose :: [Rule] -> [Fact] -> String
diagnose rules facts = case inferenceEngine rules facts of
  Nothing -> "No diagnosis could be made"
  Just (Fact _ diagnosis) -> "The diagnosis is " ++ diagnosis

Line 17: Use isJust
Found:
applyRule r wm /= Nothing
Why not:
Data.Maybe.isJust (applyRule r wm)

In [11]:
buildWorkingMemory

: 

In [2]:
-- wm :: WorkingMemory
-- wm = [ Fact "patient has fever" "yes", Fact "patient has rash" "no", Fact "patient has headache" "yes" ]

rules :: [Rule]
rules = [ Rule 1 [ Fact "patient has fever" "yes", Fact "patient has rash" "yes" ] "measles",
          Rule 2 [ Fact "patient has fever" "yes", Fact "patient has headache" "yes" ] "influenza",
          Rule 3 [ Fact "patient has headache" "yes", Fact "patient is tired" "yes" ] "flu",
          Rule 4 [ Fact "patient has sore throat" "yes", Fact "patient has fever" "yes", Fact "patient has enlarged lymph nodes" "yes" ] "mononucleosis" ]

main = do
  putStrLn $ diagnose rules wm

In [3]:
main

The diagnosis is influenza